In [0]:
import pandas as pd
import numpy as np

In [2]:
!git clone https://github.com/Team-IIT-KGP/Qspider

Cloning into 'Qspider'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 28 (delta 8), reused 20 (delta 4), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [3]:
!git clone https://github.com/kearnsw/question-type-classification

Cloning into 'question-type-classification'...
remote: Enumerating objects: 83, done.
remote: Total 83 (delta 0), reused 0 (delta 0), pack-reused 83
Unpacking objects: 100% (83/83), done.


In [4]:
# import question type dataset
df = pd.read_csv('/content/question-type-classification/data/raw/GARD.tsv', sep='\t')
df.head()

,QT,Question
0,Cause,What causes twenty-nail dystrophy?
1,PersonOrg,Can you help locate one?
2,PersonOrg,I cannot find a more affordable genetic lab th...
3,Cause,What may be the underlying cause?
4,Information,What is this condition?


In [5]:
import xml.etree.ElementTree as ET
import pandas as pd 

xtree = ET.parse('/content/Qspider/GARD_qdecomp.master.03.qtd.xml')
xroot = xtree.getroot()
rows = []
df_cols = ['Text', 'Label']
for node in xroot: 
    #print(node)
    for request in xroot.findall('Request'):
      txt = request.find('Text').text
      for q in request.findall('Question'):
        name = q.find('SubQuestion').get('qt')
      #print(txt, name)
      res = []
      res.append(txt)
      res.append(name)
      rows.append({df_cols[i]: res[i] 
                    for i, _ in enumerate(df_cols)})
GARD = pd.DataFrame(rows, columns=df_cols)
        
GARD.head()


,Text,Label
0,How can I cure my nail of anonychia congenita ...,Management
1,How can I find a physician with expertise in A...,PersonOrg
2,How can I find information on treatment for po...,Management
3,How can I find other families with children wi...,PersonOrg
4,How can I find out more about the status of re...,Information


In [6]:
GARD.shape

(588289, 2)

In [7]:
GARD.Label.value_counts()

Management        138060
Information       121953
Susceptibility     88205
Prognosis          69030
Diagnosis          47554
OtherEffect        35282
PersonOrg          30680
Cause              22243
Manifestation      19942
Other               6136
Complication        3835
NotDisease          3835
Anatomy             1534
Name: Label, dtype: int64

In [8]:
17176 + 604 + 460

18240

In [9]:
df = GARD.sample(18240)
df.head()

,Text,Label
559847,What is the life expectancy of individuals wit...,Prognosis
416563,How should patients with Birt Hogg Dube syndro...,Management
288580,I had surgery 5 years ago to remove sclerosing...,Prognosis
247209,I have a relative who has essential tremor. Ho...,Management
387542,I have acromesomelic dysplasia. As a result of...,Prognosis


In [10]:
# one-hot encode
# Get one hot encoding of Question Label (QT)
one_hot = pd.get_dummies(df['Label'])
# Drop column B as it is now encoded
df = df.drop('Label',axis = 1)
# Join the encoded df
df = df.join(one_hot)
df.head()

,Text,Anatomy,Cause,Complication,Diagnosis,Information,Management,Manifestation,NotDisease,Other,OtherEffect,PersonOrg,Prognosis,Susceptibility
559847,What is the life expectancy of individuals wit...,0,0,0,0,0,0,0,0,0,0,0,1,0
416563,How should patients with Birt Hogg Dube syndro...,0,0,0,0,0,1,0,0,0,0,0,0,0
288580,I had surgery 5 years ago to remove sclerosing...,0,0,0,0,0,0,0,0,0,0,0,1,0
247209,I have a relative who has essential tremor. Ho...,0,0,0,0,0,1,0,0,0,0,0,0,0
387542,I have acromesomelic dysplasia. As a result of...,0,0,0,0,0,0,0,0,0,0,0,1,0


In [11]:
!pip install transformers

     |████████████████████████████████| 368kB 39.3MB/s 
     |████████████████████████████████| 860kB 56.3MB/s 
     |████████████████████████████████| 1.0MB 60.6MB/s 
     |████████████████████████████████| 675kB 70.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=41c1699b0020207b3e36854feafbb65853c29e5606c18691ec1991c45f131c82
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609176 sha256=6b984e6b0e6435bbf12a8016ca26831cb863b51260d4ee0f6ed0679977505e40
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built sacremoses regex


In [12]:
!pip install seqeval


  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=cd46481578aff947e33c5ce2f4e19ece879da716f6dd231b4e017a9fd228d6db
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [13]:
!pip install tensorboardx

     |████████████████████████████████| 194kB 39.7MB/s 


In [14]:
df.columns

Index(['Text', 'Anatomy', 'Cause', 'Complication', 'Diagnosis', 'Information',
       'Management', 'Manifestation', 'NotDisease', 'Other', 'OtherEffect',
       'PersonOrg', 'Prognosis', 'Susceptibility'],
      dtype='object')

In [15]:
df['labels'] = list(zip(df.Anatomy.tolist(), df.Cause.tolist(), df.Complication.tolist(), df.Diagnosis.tolist(),  df.Information.tolist(), df.Management.tolist(),df.Manifestation.tolist(),df.NotDisease.tolist(),df.Other.tolist(),df.OtherEffect.tolist(),df.PersonOrg.tolist(),df.Prognosis.tolist(),df.Susceptibility.tolist()))
df.rename(columns={'Text':'text'}, inplace=True)
df.head()

,text,Anatomy,Cause,Complication,Diagnosis,Information,Management,Manifestation,NotDisease,Other,OtherEffect,PersonOrg,Prognosis,Susceptibility,labels
559847,What is the life expectancy of individuals wit...,0,0,0,0,0,0,0,0,0,0,0,1,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0)"
416563,How should patients with Birt Hogg Dube syndro...,0,0,0,0,0,1,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0)"
288580,I had surgery 5 years ago to remove sclerosing...,0,0,0,0,0,0,0,0,0,0,0,1,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0)"
247209,I have a relative who has essential tremor. Ho...,0,0,0,0,0,1,0,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0)"
387542,I have acromesomelic dysplasia. As a result of...,0,0,0,0,0,0,0,0,0,0,0,1,0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0)"


In [16]:
df.shape

(18240, 15)

In [17]:
from sklearn.model_selection import train_test_split

train_df = df.sample(17176)
val_df = df.sample(604)
test_df = df.sample(460)

# split all data into into train and test
#train_df, test_df = train_test_split(df, test_size=0.05)
print(train_df.shape)
print(test_df.shape)

(17176, 15)
(460, 15)


In [18]:
!pip install simpletransformers

     |████████████████████████████████| 102kB 15.0MB/s 


In [19]:
from simpletransformers.classification import MultiLabelClassificationModel


model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=13, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})

100%|██████████| 501200538/501200538 [00:13<00:00, 35901021.84B/s]


In [0]:
# Train the model
model.train_model(train_df)

Converting to features started.


Running loss: 0.109596

In [0]:
# val
result_val, model_outputs_val, wrong_predictions_val = model.eval_model(val_df)

In [0]:
# test
result, model_outputs, wrong_predictions = model.eval_model(test_df)

In [0]:
model_outputs.shape

In [0]:
result

In [0]:
# train
result_train, model_outputs_train, wrong_predictions_train = model.eval_model(train_df)

In [0]:
model_outputs_train.shape

In [0]:
# split model_outputs_train into model_outputs_train and model_outputs_val
#model_outputs_train, model_outputs_val = train_test_split(model_outputs_train, test_size=0.1)

In [0]:
model_outputs_val.shape

In [0]:
df.columns

In [0]:
# convert to dataframes
train_questiontype = pd.DataFrame(data=model_outputs_train,columns=df.columns[1:14])
train_questiontype.head()

In [0]:
val_questiontype = pd.DataFrame(data=model_outputs_val,columns=df.columns[1:14])
val_questiontype.head()

In [0]:
test_questiontype = pd.DataFrame(data=model_outputs,columns=df.columns[1:14])
test_questiontype.head()

In [0]:
# Export to csv
train_questiontype.to_csv('train_questiontype.csv')
val_questiontype.to_csv('val_questiontype.csv')
test_questiontype.to_csv('test_questiontype.csv')

In [0]:
# Qspider
dftrain=pd.read_csv("Qspider/train.tsv", sep="\t")
dfdev=pd.read_csv("Qspider/valid.tsv", sep="\t")
dftest=pd.read_csv("Qspider/test.tsv", sep="\t")
dftrain1=pd.read_csv("Qspider/trainquestiontype.csv")
dftrain1_2=pd.read_csv("train_questiontype.csv")
dfdev1=pd.read_csv("Qspider/validquestiontype.csv")
dfdev1_2=pd.read_csv("val_questiontype.csv")
dftest1=pd.read_csv("Qspider/testquestiontype.csv")
dftest1_2=pd.read_csv("test_questiontype.csv")

In [0]:
# append RoBERTa predictions onto BERT predictions
dftrain1.head()

In [0]:
dfdev1.head()

In [0]:
# replace '/t'
dftrain1.rename(columns={'PersonOrg\t': 'PersonOrg'}, inplace=True)
dfdev1.rename(columns={'PersonOrg\t': 'PersonOrg'}, inplace=True)
dftest1.rename(columns={'PersonOrg\t': 'PersonOrg'}, inplace=True)
dftrain1.head()

In [0]:
# replace 'Unnamed: 0'
dftrain1_2.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
dfdev1_2.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
dftest1_2.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
dftrain1_2.head()

In [0]:
# append
dftrain1 = dftrain1.append(dftrain1_2)
dftrain1.head()

In [0]:
dftrain1.shape

In [0]:
dfdev1 = dfdev1.append(dfdev1_2)
dfdev1.head()

In [0]:

dfdev1.shape

In [0]:
dftest1 = dftest1.append(dftest1_2)
dftest1.head()

In [0]:
print(dftrain.shape)
print(dfdev.shape)
print(dftest.shape)
print(dftrain1.shape)
print(dfdev1.shape)
print(dftest1.shape)

In [0]:
# shuffle to right data sizes
dftrain1 = dftrain1.sample(17176).reset_index(drop=True)
dfdev1 = dfdev1.sample(604).reset_index(drop=True)
dftest1 = dftest1.sample(460).reset_index(drop=True)
print(dftrain1.shape)
print(dfdev1.shape)
print(dfttest1.shape)

In [0]:
ytrainlist=list(dftrain['is_duplicate'])
ydevlist=list(dfdev['is_duplicate'])
ytestlist=list(dftest['is_duplicate'])

In [0]:
#Xtrain = dftrain1.drop('id', axis=1)
#Xtrain = dftrain1.drop('Unnamed: 0', axis=1)
Xtrain = dftrain1
# need even number of observations
#Xtrain.drop(Xtrain.tail(1).index,inplace=True)
#Xdev = dfdev1.drop('id', axis=1)
#Xdev = dfdev1.drop('Unnamed: 0', axis=1)
Xdev = dfdev1
#Xdev.drop(Xdev.tail(1).index,inplace=True)
#Xtest = dftest1.drop('Unnamed: 0', axis=1)
#Xtest = dftest1.drop('id', axis=1)
Xtest = dftest1

In [0]:
xtrain=np.array(Xtrain)
xtrain1=xtrain[:int(xtrain.shape[0]/2)]
xtrain2=xtrain[int(xtrain.shape[0]/2):]
xtrain3=np.hstack((xtrain1,xtrain2))
xdev=np.array(Xdev)
xdev1=xdev[:int(xdev.shape[0]/2)]
xdev2=xdev[int(xdev.shape[0]/2):]
xdev3=np.hstack((xdev1,xdev2))
xtest=np.array(Xtest)
xtest1=xtest[:int(xtest.shape[0]/2)]
xtest2=xtest[int(xtest.shape[0]/2):]
xtest3=np.hstack((xtest1,xtest2))

In [0]:
X_train=np.vstack((xtrain3,xdev3))
X_train.shape
y_trainlist=ytrainlist+ydevlist
len(y_trainlist)
print(len(ytrainlist))
print(len(ydevlist))

In [0]:
Xtrain5=np.copy(X_train)
for i in range(len(Xtrain5)):
    for j in range(26):
        if Xtrain5[i][j]<0.5:
            Xtrain5[i][j]=0
        else:
            Xtrain5[i][j]=1
Xtest5=np.copy(xtest3)
for i in range(len(Xtest5)):
    for j in range(26):
        if Xtest5[i][j]<0.5:
            Xtest5[i][j]=0
        else:
            Xtest5[i][j]=1
print(Xtrain5.shape)
print(Xtest5.shape)

In [0]:
import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
import time

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

dict_classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Nearest Neighbors": KNeighborsClassifier(),
    "Linear SVM": SVC(),
    "Gradient Boosting Classifier": GradientBoostingClassifier(n_estimators=10000),
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=1000),
    "Neural Net": MLPClassifier(alpha = 0.01,hidden_layer_sizes=(40, 20, 20), max_iter=50000),
    "Naive Bayes": GaussianNB()
}

def batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers = 5, verbose = True):
    """
    This method, takes as input the X, Y matrices of the Train and Test set.
    And fits them on all of the Classifiers specified in the dict_classifier.
    The trained models, and accuracies are saved in a dictionary. The reason to use a dictionary
    is because it is very easy to save the whole dictionary with the pickle module.
    
    Usually, the SVM, Random Forest and Gradient Boosting Classifier take quiet some time to train. 
    So it is best to train them on a smaller dataset first and 
    decide whether you want to comment them out or not based on the test accuracy score.
    """
    
    dict_models = {}
    for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        t_start = time.clock()
        classifier.fit(X_train, Y_train)
        t_end = time.clock()
        
        t_diff = t_end - t_start
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        Y_pred = classifier.predict(X_test)
        
        dict_models[classifier_name] = {'model': classifier, 'train_score': train_score, 'test_score': test_score, 'train_time': t_diff, 'predictions':Y_pred}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))
    return dict_models



def display_dict_models(dict_models, sort_by='test_score'):
    cls = [key for key in dict_models.keys()]
    test_s = [dict_models[key]['test_score'] for key in cls]
    training_s = [dict_models[key]['train_score'] for key in cls]
    training_t = [dict_models[key]['train_time'] for key in cls]
    
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'train_score'] = training_s[ii]
        df_.loc[ii, 'test_score'] = test_s[ii]
        df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))

In [0]:
print(Xtrain5.shape)
print(len(y_trainlist))
dict_models = batch_classify(Xtrain5, np.array(y_trainlist), Xtest5, np.array(ytestlist), no_classifiers = 11)
display_dict_models(dict_models)

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
y=dict_models['Gradient Boosting Classifier']['predictions']
print(confusion_matrix(y, ytestlist))
print(accuracy_score(y, ytestlist))